In [ ]:
import sys
sys.path.append('../')
from config import CALLHOME_ENG_10_SEC, GMM_FILE
from aux import Cluster, load_audio, int16_to_float
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import pickle5



SRC = CALLHOME_ENG_10_SEC
vad_cluster=None

"""reading ultimate gmm pkl"""
with open(GMM_FILE, 'rb') as fid:
    vad_cluster = pickle5.load(fid)
    

for i, filename in enumerate(os.listdir(SRC)):
    if i==5:
        break;
    plt.figure(figsize=(14, 5))
    print(filename)

    _, audio = load_audio(SRC + filename, 8000, mono=False)
    channel_0, channel_1 = audio[0], audio[1]
    
    _, mono = load_audio(SRC + filename, 8000, mono=True)
    
    speech_0 = vad_cluster.detect_speech(signal=channel_0, sampling_rate=8000, fit_to_audio=True)
    speech_1 = vad_cluster.detect_speech(signal=channel_1, sampling_rate=8000, fit_to_audio=True)
    mono_speech = vad_cluster.detect_speech(signal=mono, sampling_rate=8000, fit_to_audio=True)
    
    """pykaldi expect int16, not float"""



    plt.subplot(3, 1, 1)
    plt.ylim(-1., 1.)
    plt.plot(int16_to_float(channel_0))
    plt.title('channel_0', x=.5, y=.6)
    plt.fill_between(range(speech_0.shape[0]), speech_0 * .8,color='orange', alpha=0.3)

    plt.subplot(3, 1, 2)
    plt.ylim(-1., 1.)
    plt.plot(int16_to_float(channel_1))
    plt.title('channel_1', x=.5, y=.6)
    plt.fill_between(range(speech_1.shape[0]), speech_1 * .8,color='green', alpha=0.3)

    plt.subplot(3, 1, 3)
#     plt.ylim(-1., 1.)
    plt.plot(int16_to_float(mono))
    plt.title('mono', x=.6, y=.6)
    plt.fill_between(range(mono_speech.shape[0]), mono_speech * .8,color='blue', alpha=0.3)
    plt.show()
    
    # print('there is no intersection of speaker_1 and speaker_2\n\t', np.sum(speech_0 * speech_1) == 0)
